**Pre-Req:**

pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 

pip install transformers peft accelerate bitsandbytes datasets trl

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import transformers  
import bitsandbytes as bnb  


In [9]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.04s/it]


In [10]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='financial_dataset.json', split='train')

In [11]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["qkv_proj", "gate_up_proj"],
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, peft_config)

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./phi-finetuned-financial',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='epoch',
    report_to='none',
    fp16=False,
    bf16=True,
    push_to_hub=False
)

In [13]:
def formatting_func(examples):
    return examples['output']  # Directly return the text

In [14]:
import trl 
from trl import SFTTrainer
print(f"TRL version: {trl.__version__}")
print(f"SFTTrainer parameters: {SFTTrainer.__init__.__code__.co_varnames}")

TRL version: 0.19.0
SFTTrainer parameters: ('self', 'model', 'args', 'data_collator', 'train_dataset', 'eval_dataset', 'processing_class', 'compute_loss_func', 'compute_metrics', 'callbacks', 'optimizers', 'optimizer_cls_and_kwargs', 'preprocess_logits_for_metrics', 'peft_config', 'formatting_func', 'model_id', 'model_name', 'dict_args', 'eos_token', 'eos_token_id', 'chat_template_file', 'first_example', 'pad_token', 'pad_token_id', 'preprocess_dataset', 'packing', 'key', 'dataset')


In [16]:
# First tokenize your dataset
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x['output'],
        truncation=True,
        max_length=512,
        padding="max_length"
    ),
    batched=True
)

# Then create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

Truncating train dataset: 100%|██████████| 10000/10000 [00:00<00:00, 147131.91 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
# Check if model is ready for training
print(model.training)  # Should be True
print(model.device)    # Should show GPU if available

True
cuda:0


In [ ]:
trainer.train()